# Check how the particles are moving in the track 

1. Import the data correctly
2. Make it a habit to 

In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import h5py

In [2]:
# define some useful functions 

# Here I will place will the functions that Will be necessary for the ETL process. 
# ~~~~~~~ trouble shoot what the problem is with functions.py

# extract the image paths 
def extract_ImagePaths(expPath, nth_image, voltage):
    '''
    Function extracts the nth image. 
    params:
    expPath  : list of folder names that contain all the experiment, the folder before ("/..._files") folders
    nth_image: nth image whose path needs to be used.  
    returns: 
    imagePathExtract = list of two Image Paths for each experiment 
    '''
    #particle count in each experiment
    count_list = [int(x.split('\\')[-1].split('_')[0]) for i,x in enumerate(expPath)]

    # initialise and sort the data frame to finally get the correct order
    filename_data = pd.DataFrame()
    filename_data['paths'] = expPath
    filename_data['number'] = count_list

    # sort the filenames in the correct order
    filename_data = filename_data.sort_values(by = 'number')
    
    str1 = r'\*';
    str3 = r'*_files';
    str2 = str(voltage);
    A = str1 + str2 + str3;
    
    imagePathExtract = []
    for i,filePath in enumerate(filename_data['paths']):

        # extract all the experimentPaths
        experimentPath = glob.glob(filePath+A)

        # extract the images paths from the experiment Paths
        for experiment in experimentPath:
            # extract the image filenames in each experiment
            imagePath = glob.glob(experiment+'\Images\*.tif')
            # save nth image file names from the list 
            imagePathExtract.append([imagePath[nth_image]])
    return imagePathExtract 

# extract the velocity arrays from the .mat files
def Vel_data_ETL(path):
    '''
    This is a function to extract the data from VelocitiesF.mat/VelocitiesNF.mat/ 
    '''
    # create a dataframe to store the data extracted from .mat file 
    XYFPVxVy = pd.DataFrame()
    
    # extract the data from the .mat file 
    f = h5py.File(path, 'r')
    MATLAB_data = {}
    for k, v in f.items():
        MATLAB_data[k] = np.array(v)

    for i,column_name in enumerate(['X','Y','F','P','Vx','Vy']):   # I and Rg is droppped here 
        XYFPVxVy[column_name]= (MATLAB_data['XYFPVxVy'])[i];
    del(MATLAB_data) 
    f.close()
    return XYFPVxVy

# extract the densiy arrays from the .mat files 
def Den_data_ETL(path):
    '''
    This is a function to extract the data from Positions_F.mat/Positions_NF.mat
    '''
    # create a dataframe to store the data extracted from .mat file 
    XYF = pd.DataFrame()
    
    # extract the data from the .mat file 
    f = h5py.File(path, 'r')
    MATLAB_data = {}
    for k, v in f.items():
        MATLAB_data[k] = np.array(v)

    for i,column_name in enumerate(['X','Y','F']): # don't care about 'I' and 'Rg'
        XYF[column_name]= (MATLAB_data['XYF'])[i];
    del(MATLAB_data) 
    
    f.close()
    return XYF

# bring the density data and the velocity data to compute the radial binned averages 
def Transform_vel_pos_matrix(radius, dA, edges, centres_vel, centres_den, vel_path):
    '''
    This function computes single "binData" from separate entry of vel_vectors and Density data.
    IMP: Here, I am making some modifications to use the XY from the velocity data itself 
    PARAMS: 
        centres_vel: centre of the images used for velocity computations
        centres_den: centre of the images used for density computations
        den_path:    path of the density matrix (.mat)
        vel_path:    path of e velocity matrix  (.mat)
        bins:        number of bins
        conf_rad:    radius of the circular confinment in px.
    
    RETURNS:
        bindata: data with "R, theta, V0, V_phi, V_r, P_phi , P_r, Pi,rho, Ar_frac""
    '''
    
    # Extract the matrix for the velocity      
    vel_vectors = Vel_data_ETL(vel_path)

    # centre of the velocity images  
    centres_vel = np.array(centres_vel)

    # centre of the density images
    centres_den = np.array(centres_den)  

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Radial profiles ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ velocity vector computations ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # compute the Radial distances.
    vel_vectors['R'] = np.sqrt((vel_vectors['X'] -centres_vel[0]) **2  + (vel_vectors['Y'] - centres_vel[1])**2)
    # compute the theta using the relatve position of the particle to the vortex centre 
    vel_vectors['theta'] = np.arctan2((vel_vectors['Y']-centres_vel[1]), (vel_vectors['X']-centres_vel[0]))
    
    # measure the total number of frames
    total_frames = max(vel_vectors['F'])
    # clear the memory of the X and Y values 
    vel_vectors.drop(columns = ['X','Y','P'], inplace=True)
    
    # compute the propulsion speed
    vel_vectors['V0'] = np.sqrt(vel_vectors['Vx']**2 +vel_vectors['Vy']**2)
    # compute the radial and the azimuthal velocities unit vectors 
    vel_vectors['P_phi'] = (-vel_vectors['Vx']*np.sin(vel_vectors['theta']) + vel_vectors['Vy']*np.cos(vel_vectors['theta']))/vel_vectors['V0']
    vel_vectors['P_r'] = (vel_vectors['Vx']*np.cos(vel_vectors['theta']) + vel_vectors['Vy']*np.sin(vel_vectors['theta']))/vel_vectors['V0']
    
    # nematic order parameters computations
    vel_vectors['P_phi_P_phi'] = vel_vectors['P_phi']*vel_vectors['P_phi']
    vel_vectors['P_r_P_r']     = vel_vectors['P_r']*vel_vectors['P_r']
    vel_vectors['P_r_P_phi']   = vel_vectors['P_phi']*vel_vectors['P_r']
    
    # clear the memory of the Vx and Vy , Vphi and Vr values 
    #vel_vectors.drop(columns = ['V_phi', 'V_r'], inplace=True)
    
    # create another column with the BIN IDs
    vel_vectors['binID'] = np.digitize(vel_vectors['R'], edges, right=True)
    

    # develop and intantiate the bins [do this too avoid referencing complications]
    #binData = pd.DataFrame()
    # create  the 'binData' from the velocity vector data. 
    binData = vel_vectors.groupby(by=['binID']).mean()
    
    # Compute the mean polarisation in each bin 
    binData['Pi'] = np.sqrt((binData['P_r'])**2 + (binData['P_phi'])**2)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ density computations ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    # extract the matrix for the density
    XYF = pd.DataFrame()
    XYF['R'] = vel_vectors['R']
    XYF['F'] = vel_vectors['F']
    XYF['binID'] = np.digitize(XYF['R'], edges, right=True)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ add the rho to the bin Data ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # compute the count sizes
    binData['rho'] = XYF.groupby(by=['binID']).size()

    # cnsider only the first n bins that you require...
    binData = binData[:len(dA)]
    
    binData['rho'] = binData['rho']/(total_frames*dA)
    
    # compute the average density of experiment
    average_density = len(XYF)/total_frames
    # compute the Area Fractions
    binData['Ar_frac'] = binData['rho']*(np.pi*radius**2) 
    # return the stuff
    return binData, average_density, vel_vectors

# build the final path to the density and velocity files 
def part_type_path_list(particle, den_expPath, velocity_expPath):
    '''
    This function builds the path to the density and velocity files 
    '''
    # choose the particle type
    if particle == 'F':
        denPathExt = r'\**\Analysis\1_Positions\Positions_F.mat'
        velPathExt = r'\**\Analysis\3_Velocities\VelocitiesF.mat'
        particle_rad = 5 #um
    elif particle =='NF':
        denPathExt = r'\**\Analysis\1_Positions\Positions_NF.mat'    
        velPathExt = r'\**\Analysis\3_Velocities\VelocitiesNF.mat'
        particle_rad = 3.5 #um
        
    # Extract the matrix for the velocity      
    VelFilePath = glob.glob(velocity_expPath+velPathExt)

    # extract the matrix for the density
    DenFilePath = glob.glob(den_expPath+denPathExt)
    return DenFilePath, VelFilePath

def part_type_path_list_fragmented(fragment, particle, den_expPath, velocity_expPath):
    '''
    This function builds the path to the density and velocity files 
    Here, we pick up the fragments of the same confiment (for 7um)
    becoz, the along r within the same ROI, i could not capture the entire radial distribution
    '''
    if (fragment == 'A'):
    
        # choose the particle type
        if particle == 'F':
            denPathExt = r'\*A_files\Analysis\1_Positions\Positions_F.mat'
            velPathExt = r'\*A_files\Analysis\3_Velocities\VelocitiesF.mat'
            particle_rad = 5 #um
        elif particle =='NF':
            denPathExt = r'\*A_files\Analysis\1_Positions\Positions_NF.mat'    
            velPathExt = r'\*A_files\Analysis\3_Velocities\VelocitiesNF.mat'
            particle_rad = 3.5 #um
    
    else:
        # choose the particle type
        if particle == 'F':
            denPathExt = r'\*B_files\Analysis\1_Positions\Positions_F.mat'
            velPathExt = r'\*B_files\Analysis\3_Velocities\VelocitiesF.mat'
            particle_rad = 5 #um
        elif particle =='NF':
            denPathExt = r'\*B_files\Analysis\1_Positions\Positions_NF.mat'    
            velPathExt = r'\*B_files\Analysis\3_Velocities\VelocitiesNF.mat'
            particle_rad = 3.5 #um
    
    # Extract the matrix for the velocity      
    VelFilePath = glob.glob(velocity_expPath+velPathExt)

    # extract the matrix for the density
    DenFilePath = glob.glob(den_expPath+denPathExt)
    return DenFilePath, VelFilePath

In [3]:
bins = 18
confinement_radius = 1025 #px
vel_centre = [1176, 1028]
den_centre = [1176, 1028]
edges = np.linspace(1,confinement_radius,(bins+1));

# define the edges and the elementary area and the central value of R
dA =  np.pi*(edges[1:]**2 - edges[:-1]**2)

#~~~~~~~~~~~~ Store the binData tables as lists ~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~ compute for 'NF' particles ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ declare the master paths ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
path = r'D:\Ring formation in flocks\2_files\Analysis\3_Velocities\VelocitiesNF.mat'
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
particle = 'NF'
radius = 3.5 #um 
# obtain the final bin matrix
[bindata_NF, avg_density_NF,vel_data_NF] = Transform_vel_pos_matrix(radius,dA, edges, vel_centre, den_centre, path)

In [6]:
min(vel_data_NF['theta'])

-3.1405722457806884

# Do the following things:
1. extract only a part of the confinement which has a clear track formation.
2.
3.